# LangChain: Memory(메모리, 기억)

## 목차
* ConversationBufferMemory
* ConversationBufferWindowMemory
* ConversationTokenBufferMemory
* ConversationSummaryMemory

## ConversationBufferMemory
우리가 이전까지 다뤘던 것은 하나의 프롬프트에 대해서만 답변하는 방식이었습니다.  
하지만 채팅 형식의 가장 큰 장점은 이전의 정보를 context로 기억하고 이에 기반한 자연스러운 대화겠죠.  
이를 위해서는 <span style='color:red'>이전 대화를 '기억'</span>하는 것이 필요한데, 이 '기억'과 관련된 여러 기능을 제공하고 있습니다.

In [ ]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

import warnings
warnings.filterwarnings('ignore')

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory


여기에서 llm의 종류를 지정해주는 이유는 토큰 단위때문이기도 한데, 모델마다 입력 처리 방식이 다를 수 있기 때문입니다.

In [ ]:
llm = ChatOpenAI(temperature=0.0)
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

In [ ]:
conversation.predict(input="Hi, my name is Andrew")

In [ ]:
conversation.predict(input="What is 1+1?")

In [ ]:
conversation.predict(input="What is my name?")

In [ ]:
print(memory.buffer)

In [ ]:
memory.load_memory_variables({})

In [ ]:
memory = ConversationBufferMemory()

`save_context` 함수를 이용하면 프롬프트와 답변을 모델에 메모리에 저장할 수 있습니다.

In [ ]:
memory.save_context({"input": "Hi"}, 
                    {"output": "What's up"})

In [ ]:
print(memory.buffer)

반대로 메모리에 저장된 변수들이나 내용을 확인하는 함수도 존재합니다.

In [ ]:
memory.load_memory_variables({})

In [ ]:
memory.save_context({"input": "Not much, just hanging"}, 
                    {"output": "Cool"})

In [ ]:
memory.load_memory_variables({})

## ConversationBufferWindowMemory
윈도우는 일정 부분에 대한 기능을 표현할 때 자주 등장하는 단어입니다.  
여기서는 <span style='color:red'>몇 개 단위의 대화를 기억할지를 k라는 변수로 설정</span>할 수 있습니다.  
k=1인 경우, 직전의 대화만 메모리에 저장해두겠다는 의미가 됩니다.

In [ ]:
from langchain.memory import ConversationBufferWindowMemory

In [ ]:
memory = ConversationBufferWindowMemory(k=1)               

두 개의 대화를 저장한다고 하더라도 메모리 변수에 담긴 것은 한 개의 대화뿐임을 알 수 있습니다.

In [ ]:
memory.save_context({"input": "Hi"},
                    {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})


In [ ]:
memory.load_memory_variables({})

In [ ]:
llm = ChatOpenAI(temperature=0.0)
memory = ConversationBufferWindowMemory(k=1)
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=False
)

In [ ]:
conversation.predict(input="Hi, my name is Andrew")

In [ ]:
conversation.predict(input="What is 1+1?")

In [ ]:
conversation.predict(input="What is my name?")

## ConversationTokenBufferMemory
이 함수를 사용하면 <span style='color:red'>기억할 수 있는 최대 토큰의 개수를 지정</span>할 수 있습니다.  
`max_token_limit` 변수에 다른 값을 할당하면서 메모리에 어떤 정보가 남아 있는지 확인해보세요.

In [ ]:
#!pip install tiktoken

In [ ]:
from langchain.memory import ConversationTokenBufferMemory
from langchain.llms import OpenAI
llm = ChatOpenAI(temperature=0.0)

In [ ]:
memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=30)
memory.save_context({"input": "AI is what?!"},
                    {"output": "Amazing!"})
memory.save_context({"input": "Backpropagation is what?"},
                    {"output": "Beautiful!"})
memory.save_context({"input": "Chatbots are what?"}, 
                    {"output": "Charming!"})

In [ ]:
memory.load_memory_variables({})

## ConversationSummaryMemory
이 함수를 사용하면 요약 정보를 메모리에 저장할 수 있습니다.  
<span style='color:red'>사용자가 이전에 어떤 요청을 했는지가 시스템 메세지에 요약되어 추가</span>된 것을 확인해 보세요.

In [ ]:
from langchain.memory import ConversationSummaryBufferMemory


In [ ]:
# create a long string
schedule = "There is a meeting at 8am with your product team. \
You will need your powerpoint presentation prepared. \
9am-12pm have time to work on your LangChain \
project which will go quickly because Langchain is such a powerful tool. \
At Noon, lunch at the italian resturant with a customer who is driving \
from over an hour away to meet you to understand the latest in AI. \
Be sure to bring your laptop to show the latest LLM demo."

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)
memory.save_context({"input": "Hello"}, {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})
memory.save_context({"input": "What is on the schedule today?"}, 
                    {"output": f"{schedule}"})

In [ ]:
memory.load_memory_variables({})

In [ ]:
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

In [ ]:
conversation.predict(input="What would be a good demo to show?")

In [ ]:
memory.load_memory_variables({})